In [1]:
import os
import urllib.request
import re

In [2]:
file_path = "the-verdict.txt"

if not os.path.isfile(file_path):
    url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
    print("Downloading data :", file_path, "\nat :", url)
    urllib.request.urlretrieve(url, file_path)


In [3]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters :", len(raw_text))
print("type :", type(raw_text))
print("sample:", raw_text[:86])

Total number of characters : 20479
type : <class 'str'>
sample: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--


In [4]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [i.strip() for i in preprocessed if i.strip()]
print(len(preprocessed), preprocessed[:10])

4690 ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius']


In [5]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [6]:
vocab = {str: id for id, str in enumerate(all_words)}

In [7]:
class TokenizerV1:
    def __init__(self, vocab: dict[str, int]):
        self.str_to_int: dict[str, int] = vocab
        self.int_to_str: dict[int, str] = {id: str for str, id in vocab.items()}

    def encode(self, text: str):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [i.strip() for i in preprocessed if i.strip()]

        preprocessed = [i if i in self.str_to_int else "<|unk|>" for i in preprocessed]
        
        ids = [self.str_to_int[str] for str in preprocessed]

        return ids

    def decode(self, ids: list[int]):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)

        return text

In [8]:
tokenizer = TokenizerV1(vocab)
text = "great surprise to me to hear that, in the height of"
tokenizer.decode(tokenizer.encode(text))

'great surprise to me to hear that, in the height of'

In [9]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: id for id, token in enumerate(all_tokens)}
print(len(vocab))

1132


In [34]:
text1 = "Hello."    # unseen word
text2 = "Cool dog." # unseen words
text = "<|endoftext|>".join([text1, text2])
text

'Hello.<|endoftext|>Cool dog.'

In [35]:
tokenizer = TokenizerV1(vocab)
print(tokenizer.encode(text), "->", tokenizer.decode(tokenizer.encode(text)))

[1131, 7, 1131, 1131, 7] -> <|unk|>. <|unk|> <|unk|>.


---

In [36]:
import tiktoken

In [37]:
tokenizer = tiktoken.get_encoding("gpt2")

In [38]:
print("text:", text)
tokens = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("encoded tokens:", tokens)
strings = tokenizer.decode(tokens)
print("decoded tokens:", strings)

text: Hello.<|endoftext|>Cool dog.
encoded tokens: [15496, 13, 50256, 34530, 3290, 13]
decoded tokens: Hello.<|endoftext|>Cool dog.


In [42]:
print("- Example for an unkown word, it decomposes the word into subwords")
print(tokenizer.encode("Akwirw ier"))
for i in tokenizer.encode("Akwirw ier"):
    print(i, tokenizer.decode([i]))
print(tokenizer.decode(tokenizer.encode("Akwirw ier")))

- Example for an unkown word, it decomposes the word into subwords
[33901, 86, 343, 86, 220, 959]
33901 Ak
86 w
343 ir
86 w
220  
959 ier
Akwirw ier


---

In [44]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [46]:
enc_sample = enc_text[50:]

In [51]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f'{x=}')
print(f'{y     =}')

x=[290, 4920, 2241, 287]
y     =[4920, 2241, 287, 257]


In [58]:
print('[input] ---> [output]')
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

[input] ---> [output]
 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


---

In [59]:
import torch
from torch.utils.data import Dataset, DataLoader

In [68]:
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i+1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [69]:
def create_dataloader(txt, batch_size=4, max_length=256,
                     stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [75]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(raw_text, batch_size=1, max_length=4,
                     stride=1, shuffle=False, drop_last=True, num_workers=0)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [77]:
# with a larger batch_size
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4,
                     stride=4, shuffle=False, drop_last=True, num_workers=0)
data_iter = iter(dataloader)

inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


---

### Embedding

In [101]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3 # higher value -> more precision

In [102]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer, embedding_layer.weight)

Embedding(6, 3) Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [103]:
print("embedding vector for token_id=3:\n", embedding_layer(torch.tensor([3])))

embedding vector for token_id=3:
 tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [104]:
print("embedding vectors for tokens=[2,3,5,1]:\n", embedding_layer(input_ids))

embedding vectors for tokens=[2,3,5,1]:
 tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [108]:
vocab_size = tokenizer.n_vocab # 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [114]:
batch_size = 8
max_length = 4
dataloader = create_dataloader(raw_text, batch_size=batch_size, max_length=max_length,
                     stride=max_length, shuffle=False, drop_last=True, num_workers=0)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [115]:
token_embedding = token_embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [117]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer
